In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, MaxPooling2D,Flatten, Dense, Dropout, AveragePooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from utils import *
import numpy as np

import cv2


In [3]:
# Data Path
with_mask_paths = ["./data/data1/with_mask/with_mask_" + str(i) + ".jpg" for i in range(1, 4500 + 1)]
without_mask_paths = ["./data/data1/without_mask/without_mask_" + str(i) + ".jpg" for i in range(1, 4500 + 1)]


In [4]:
# Declare array for storing data & corresponding label
data = []
label = []

# Load with_mask data
load_data_with_label(path_arr=with_mask_paths,
                     data_arr=data,
                     label_arr=label,
                     img_size=(224, 224, 3),
                     _label = 1)


In [5]:

# Load without_mask data
load_data_with_label(path_arr=without_mask_paths,
                     data_arr=data,
                     label_arr=label,
                     img_size=(224, 224, 3),
                     _label = 0)

print("Data Load Done!")

Data Load Done!


In [6]:

# Convert to numpy array
data = np.array(data, dtype="float32")
label = np.array(label)

# Label one-hot encoding
_lb = LabelBinarizer()

label = _lb.fit_transform(label)
label = to_categorical(label)


In [7]:

# Split data to train data and test data
(train_X, test_X, train_Y, test_Y) = train_test_split(data, label,
                                                      test_size=0.15,
                                                      random_state=777,
                                                      shuffle=True)


In [8]:

# Make CNN Model
CNN = Sequential([
    Input(shape=(224, 224, 3)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    Conv2D(32, (3, 3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    Conv2D(64, (3,3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    Conv2D(128, (3,3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    Conv2D(128, (3,3), padding='SAME', activation='relu',
           kernel_regularizer=l2(5e-4)),
    MaxPooling2D((2,2)),

    Conv2D(256, (1, 1), activation='relu'),
    Dropout(0.5),
    Conv2D(128, (1,1), activation='relu'),
    AveragePooling2D(pool_size=(3, 3)),

    Flatten(),
    Dropout(0.5),
    Dense(2, activation='softmax')

])

In [9]:
# Declare epoch, batch size, optimizer
epochs = 30
batch_size = 100
optimizer = Adam(learning_rate=1e-4, decay = 1e-4/epochs)



In [10]:
# Compile and training
CNN.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics=['accuracy'])

history = CNN.fit(train_X, train_Y,
                    batch_size=batch_size,
                    validation_split=0.1,
                    epochs = epochs)

# Model save
CNN.save("./checkpoint")

ResourceExhaustedError: OOM when allocating tensor with shape[6480,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:GatherV2]

In [ ]:
# Draw Plot and Save
training_log_plot(history)

# Evaluate
CNN.evaluate(test_X, test_Y)

export_model_summary(CNN, "LightweightCNN")
